100%|█████████████████████████████████████████████████████████████████████████████| 8.91k/8.91k [00:00<00:00, 9.10MB/s]

Extracting files...
Path to dataset files: C:\Users\Administrator\.cache\kagglehub\datasets\akshaydattatraykhare\diabetes-dataset\versions\1
